# Orbital-Optimization for reducing the one-norm

Author : Dr. Saad Yalouz - MSc Emiel Koridon (Cool titles)

A starting point to minimize the One-norm using Orbital-Optimization and a gradient free optimizer

## Theory 
The idea here is to employ a unitary transformation to modify the MOs of the system in order to minimize the One-norm. To build up this unitary, we start by the definition of the Orbital rotation operator which reads
$$ U(\vec{\kappa}) =\sum_{t>u}^{active} \kappa_{tu} (\hat{E}_{tu} - \hat{E}_{ut})$$
with $\hat{E}_{tu}=\frac{1} {\sqrt{2}}\sum_{\sigma}a^\dagger_{t,\sigma} a_{u,\sigma}$ and $\kappa_{tu}$ a rotation parameter affecting MOs $t$ and $u$ (note the absence of self rotation here with $t\neq u$). This operator takes the form of single excitations (with $t,u$ indices running only in the active space). Starting from this definition, a little bit of aglebra can show that implementing this rotation operator on creation/anihilation operators is strictly equivalent to redefining the MO coeff matrix of the Hamiltonian as follows
$$ \mathbf{C}_{new}(\vec{\kappa}) = \mathbf{C}_{ref} \mathbf{U}_{OO}(\vec{\kappa}) $$ 
Here, $\mathbf{U}_{OO}(\vec{\kappa})$ is a matrix representation of the effect of the rotations operator on the MOs. This matrix is an exponentiated version of the skew-matrix noted $\mathbf{K}$ encoding the parameter $\vec{\kappa}$. In other words
$$\mathbf{U}_{OO}(\vec{\kappa}) = e^{-\mathbf{K}} \text{,  with  } \mathbf{K} = skew(\vec{\kappa}) = \pmatrix{ 0 & \kappa_{01} & \kappa_{02} & \ldots & \kappa_{0N} \\
-\kappa_{01} & 0 & \kappa_{12} & \ldots & \kappa_{1N} \\
-\kappa_{02} & -\kappa_{12} & 0 &\ldots & \vdots \\
\vdots & \vdots & \vdots &\ddots & \vdots \\
-\kappa_{0N} & -\kappa_{1N} & \ldots & \ldots & 0 }$$

## Numerical implementation

Starting from this, we can encode this exponential numerically and use the kappa parameters (total number of parameters = $N(N-1)/2$) as degrees of freedom to reduce the one-norm. For this, first I use scimpy symbols to encode the matrix.

In [10]:
import psi4
import sys
import os
import openfermion
from openfermionpyscf import run_pyscf, compute_integrals
import numpy as np
import copy
from scipy.optimize import minimize
import math as m
import scipy 
import sympy
from sympy.matrices import zeros
from random import random
from openfermion import ops, InteractionOperator, get_sparse_operator, jordan_wigner
from openfermion.functionals import JW1norm_spat
from pyscf import gto, scf, lo, tools, fci, ao2mo, mcscf, df
import module as md
import time

# Set molecule parameters.
basis = ['sto-3g', 'cc-pvdz','def2svp'][1]
multiplicity = 1

# Set calculation parameters.
consider_cas = 1 # Do we consider an active space or the full space?
# Set size of active space
n_orbitals = 12
n_electrons = 12
# Choose whether you want to start from localized orbitals or canonical orbitals
localize = 0
# Choose whether you want to start with a random initial rotation
randomize = 1
# Import .xyz geometry. This is kind of an involved code because I have many different geometries available...
# If you want to consider a Hydrogen chain or ring, set the following to 1
H_chain = 0
H_ring = 0
run_fci = 0
run_casci = 0
if H_chain:
    N_atoms = 14
    r = 1.8
    if consider_cas:
        description = 'H' + str(N_atoms) + str(basis) + 'ne' + str(n_electrons) +\
            'no' + str(n_orbitals)
    else:
        description = 'H' + str(N_atoms) + str(basis)
    geometry = [('H',( 0., 0., z*r)) for z in range(N_atoms)]
    
elif H_ring:
    theta = 88 * np.pi/180
    r = 1.3
    # Introduction of the molecular structure (.txt file)
    geometry = [
                ('H', (r*np.cos(theta/2.),   r*np.sin(theta/2.),  0.)),
                ('H', (r*np.cos(theta/2.),   -r*np.sin(theta/2.), 0.)),
                ('H', (-r*np.cos(theta/2.),  r*np.sin(theta/2.),  0.)),
                ('H', (-r*np.cos(theta/2.),  -r*np.sin(theta/2.), 0.))
                ]
    if consider_cas:
        description = 'H4' + str(basis) + 'ne' + str(n_electrons) + 'no' + str(n_orbitals)
    else:
        description = 'H4' + str(basis)

else:
    fname = ['xyz_files/H2nosym.txt','xyz_files/H2COnosym.txt','xyz_files/H10.txt',\
             'xyz_files/C2.txt', 'xyz_files/LiH.txt', 'xyz_files/HLiO.txt', \
             'xyz_files/H2Onosym.txt', 'xyz_files/H14.txt', \
             'xyz_files/hnch2_s0min_dzp.txt', 'xyz_files/hnc3h6_s0min_dzp.txt',\
             'xyz_files/hnc5h10_s0min_dzp.txt', 'xyz_files/hnc7h14_s0min_dzp.txt',\
             'xyz_files/benzene.txt','xyz_files/PCy3.txt','xyz_files/PCy3Cl2Ru.txt'][8]
    geometry = md.xyz_to_geom(fname)
    if consider_cas:
        description = fname.replace('xyz_files/','').replace('.txt','') + str(basis)\
            + 'ne' + str(n_electrons) + 'no' + str(n_orbitals)
    else:
        description = fname.replace('xyz_files/','').replace('.txt','') + str(basis)

# Determine number of AOs and electrons
nao = md.count_ao(geometry,basis,spin=multiplicity-1)
nelec = md.count_elec(geometry,basis,spin=multiplicity-1)
nmo = nao
print('nao:',nao,'\nnelec:', nelec)
threshold = 1e-10

# Set active indices
if consider_cas:
    ncore = (nelec - n_electrons) // 2
    ntot = ncore + n_orbitals # Number of core orbitals + number of active orbitals
    active_indices = list(range(ncore,ntot))
    occupied_indices = list(range(ncore))
else:
    ncore = 0
    ntot = nmo
    active_indices = list(range(nmo))

print(active_indices,occupied_indices)
# Build molecule and run RHF
mol = gto.Mole()
mol.atom = geometry
mol.basis = basis
mol.spin = multiplicity - 1
mol.symmetry = 0
mol.build()

if multiplicity == 1:
    myhf = scf.RHF(mol)
else:
    myhf = scf.ROHF(mol)

myhf.run()

# Extract MO_coeff and integrals. Then determine qubit 1-norm for CMOs and Pipek-Mezey MOs.

C_nonloc = np.copy(myhf.mo_coeff)

constant = float(mol.energy_nuc())


print('---------NON-LOCALIZED_ORBITALS---------')

C_copy = np.copy(C_nonloc)
one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_copy[:,:ntot], threshold)
print("1bdyint shape:",one_body_integrals.shape)
if consider_cas:
    CASconstant, one_body_integrals, two_body_integrals =\
    md.get_active_space_integrals(one_body_integrals,
                                  two_body_integrals,
                                  occupied_indices,
                                  active_indices)
else:
    CASconstant = 0
t4 = time.time()
qub1norm = JW1norm_spat(np.copy(constant+CASconstant),
                        np.copy(one_body_integrals),
                        np.copy(two_body_integrals))
print('\ncalculating norm of qubit hamiltonian took', time.time()-t4)

print('\n')

print('---------LOCALIZED_ORBITALS---------')
localizemethod = 'Pipek-Mezey'

C = np.copy(C_nonloc)
if localizemethod == 'Pipek-Mezey':
    orb = lo.PipekMezey(mol).kernel(C[:,ncore:ntot])
elif localizemethod == 'Boys':
    orb = lo.Boys(mol).kernel(C[:,ncore:ntot])
elif localizemethod == 'ibo':
    orb = lo.ibo.ibo(mol, C[:,ncore:ntot])
elif localizemethod == 'ER':
    orb = lo.EdmistonRuedenberg(mol).kernel(C[:,ncore:ntot])
else:
    raise ValueError('Localization method not recognized')

C_locPM = np.hstack((C_nonloc[:,:ncore],orb,C_nonloc[:,ntot:nmo]))
one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_locPM[:,:ntot], threshold)
if consider_cas:
    CASconstant, one_body_integrals, two_body_integrals =\
    md.get_active_space_integrals(one_body_integrals,
                                  two_body_integrals,
                                  occupied_indices,
                                  active_indices)
else:
    CASconstant = 0
t4 = time.time()
qub1norm_locPM = JW1norm_spat(np.copy(constant+CASconstant),
                        np.copy(one_body_integrals),
                        np.copy(two_body_integrals))
print('calculating norm of qubit hamiltonian took', time.time()-t4)

if consider_cas:
    print("Considering 1-norm of active space Hamiltonian")
else:
    print("Considering 1-norm of full space Hamiltonian")
    
print("starting 1-norm nonloc is:",qub1norm)
print("1norm locPM is:",qub1norm_locPM)


OPT_OO_MAX_ITER  = 50000 # Maximum number of steps for the OO

# Building the K matrix : generator for the MO rotations 
Rot_param_values = []
Rot_param_names  = []
bounds = (-1.,1.)

K_mat_sym = zeros( nmo )
bounds = []
for q in range(nmo-1):
    for p in range(q+1,nmo):
        if ( p in active_indices and q in active_indices ):
            K_mat_sym[ p, q ] = - sympy.Symbol( "K_{}{}".format( p, q ) )
            K_mat_sym[ q, p ] = - K_mat_sym[ p, q ] 
            Rot_param_names  += [sympy.Symbol( "K_{}{}".format( p, q ))]
            Rot_param_values += [1e-1 * (random()-0.5) if randomize else 0.] # <== Set starting Rot_param_values here.
            bounds += [ [-1., 1.] ] 

# Building the bounds for the rotational parameter amplitudes in the form of constraints
# (simply beacause cobyla doesn't have intrinsic bound options)
cons = []
for factor in range(len(bounds)):
    lower, upper = bounds[factor]
    l = {'type': 'ineq',
         'fun': lambda x, lb=lower, i=factor: x[i] - lb}
    u = {'type': 'ineq',
         'fun': lambda x, ub=upper, i=factor: ub - x[i]}
    cons.append(l)
    cons.append(u)


nao: 43 
nelec: 16
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] [0, 1]
converged SCF energy = -94.0392279960234
---------NON-LOCALIZED_ORBITALS---------
1bdyint shape: (14, 14)

calculating norm of qubit hamiltonian took 0.018728256225585938


---------LOCALIZED_ORBITALS---------
calculating norm of qubit hamiltonian took 0.017575502395629883
Considering 1-norm of active space Hamiltonian
starting 1-norm nonloc is: 163.4113636170075
1norm locPM is: 144.0570143662687


In [11]:
print(CASconstant)
print(active_indices)
print(occupied_indices)
print(ncore,ntot)
print(Rot_param_values)

-90.85474342116778
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[0, 1]
2 14
[0.010255815794448932, -0.03237538727216732, -0.03178822806563725, -0.02991582525313472, 0.03019003363790399, 0.007408783720392387, -0.04388962918488357, 0.024680185238607932, -0.015906011214042105, 0.018040757594735803, -0.0049961985817223, 0.04385858933658868, 0.012902763038535991, 0.020930130821420503, -0.019925864997159416, 0.0026175949168891323, 0.011128539137352468, -0.03365412815944886, 0.04250373280661501, -0.02217466760343008, -0.01882236598693615, -0.020067260279926426, 0.03653337102355088, 0.04142119791861503, 0.0005280924392237485, -0.04873896086573343, 0.014443458795462228, 0.04117518498264216, -0.006531863478817391, 0.01677472703142201, 0.026865995832495028, 0.0003336240429502091, 0.034441163138694234, -0.02854617894147914, -0.01328243228922369, 0.03585785957441994, 0.024056117985280803, 0.014140183723928468, -0.009719075939639067, 0.01393778430948548, -0.019244940326184557, 0.02194890823578907, 0.040

In [6]:
K      = np.array( K_mat_sym.subs(list(zip(Rot_param_names,Rot_param_values))) ).astype(np.float64)  
U_OO   = scipy.linalg.expm( - K )

print("check if U_OO is unitary:",np.allclose(np.eye(U_OO.shape[0]), U_OO @ U_OO.T))

check if U_OO is unitary: True


Then I define a cost function to encode the evaluation of the One-Norm using the new MO coeff matrix. In the latter, we compute the 1-norm from the integrals. To proceed, we use $\mathbf{C}_{new}(\vec{\kappa})$ to build up the new electronic integrals.

In [2]:
def K_matr(Rot_param_values, active_indices):
    K = zeros( nmo )
    n = 0
    for q in range(nmo-1):
        for p in range(q+1,nmo):
            if ( p in active_indices and q in active_indices ):
                K[ p, q ] = - Rot_param_values[n]
                K[ q, p ] = - K[ p, q ]
                n += 1
    return np.array(K)

def Cost_function_OO_OneNorm(Rot_param_values):
#                              Rot_param_names,
#                              K_matr):
    """
    Cost function to minimize the One-Norm using MO rotations.
    """
    t6 = time.time()
    t5 = time.time()
    K = K_matr(Rot_param_values, active_indices)
    print("\nInitializing K-matrix took:",time.time()-t5)
    t1 = time.time()
    U_OO   = scipy.linalg.expm( - K )
    print("exponentiating matrix took:",time.time()-t1)
    t3 = time.time()
    if localize:
        C_MO   = C_locPM @ U_OO
    else:
        C_MO   = C_nonloc @ U_OO
    print("Matrix multiplication took:",time.time()-t3)
    t2 = time.time()
    one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_MO[:,:ntot], threshold)
    print("extracting integrals took:",time.time()-t2)
    t4 = time.time()
    if consider_cas:
        CASconstant, one_body_integrals, two_body_integrals =\
        md.get_active_space_integrals(one_body_integrals,
                                      two_body_integrals,
                                      occupied_indices,
                                      active_indices)
    else:
        CASconstant = 0
    t4 = time.time()
    OneNorm = JW1norm_spat(np.copy(constant+CASconstant),
                           np.copy(one_body_integrals),
                           np.copy(two_body_integrals))
    print('calculating norm of qubit hamiltonian took', time.time()-t4)
    print('1-norm is:',OneNorm)
    print("total time for 1 cost function evaluation is", time.time()-t6)
    return OneNorm

print("starting 1-norm nonloc is:",qub1norm)
print("1norm locPM is:",qub1norm_locPM)
t7 = time.time()
f_min_OO = minimize( Cost_function_OO_OneNorm,
                      x0      = Rot_param_values,
                      constraints=cons,
                      method  = 'cobyla',
                      options = {'maxiter': OPT_OO_MAX_ITER,
                                'tol'    : 1e-5,
                                'disp': True}  )
print("total time for minimization with",OPT_OO_MAX_ITER,"max iterations was:", time.time()-t7)

starting 1-norm nonloc is: 163.4113636170075
1norm locPM is: 144.0570143662687

Initializing K-matrix took: 0.01215505599975586
exponentiating matrix took: 0.0030961036682128906
Matrix multiplication took: 7.963180541992188e-05
extracting integrals took: 0.08914518356323242
calculating norm of qubit hamiltonian took 0.017635822296142578
1-norm is: 168.2746380065764
total time for 1 cost function evaluation is 0.12297439575195312

Initializing K-matrix took: 0.015580177307128906
exponentiating matrix took: 0.0034151077270507812
Matrix multiplication took: 8.654594421386719e-05
extracting integrals took: 0.1199951171875
calculating norm of qubit hamiltonian took 0.01750493049621582
1-norm is: 168.71670009132083
total time for 1 cost function evaluation is 0.1577003002166748

Initializing K-matrix took: 0.015434503555297852
exponentiating matrix took: 0.0032672882080078125
Matrix multiplication took: 2.2649765014648438e-05
extracting integrals took: 0.12950348854064941
calculating norm of

exponentiating matrix took: 0.0035948753356933594
Matrix multiplication took: 2.4557113647460938e-05
extracting integrals took: 0.11985015869140625
calculating norm of qubit hamiltonian took 0.017920255661010742
1-norm is: 173.37775217290934
total time for 1 cost function evaluation is 0.15829920768737793

Initializing K-matrix took: 0.015491962432861328
exponentiating matrix took: 0.0032808780670166016
Matrix multiplication took: 2.3603439331054688e-05
extracting integrals took: 0.11840343475341797
calculating norm of qubit hamiltonian took 0.01798105239868164
1-norm is: 167.76580102640867
total time for 1 cost function evaluation is 0.15642929077148438

Initializing K-matrix took: 0.017687320709228516
exponentiating matrix took: 0.0033147335052490234
Matrix multiplication took: 2.4318695068359375e-05
extracting integrals took: 0.12181687355041504
calculating norm of qubit hamiltonian took 0.017822265625
1-norm is: 173.58061018204833
total time for 1 cost function evaluation is 0.1621

extracting integrals took: 0.1222083568572998
calculating norm of qubit hamiltonian took 0.0184633731842041
1-norm is: 165.60011862548598
total time for 1 cost function evaluation is 0.16066622734069824

Initializing K-matrix took: 0.01735830307006836
exponentiating matrix took: 0.003361940383911133
Matrix multiplication took: 2.3365020751953125e-05
extracting integrals took: 0.1330585479736328
calculating norm of qubit hamiltonian took 0.018421411514282227
1-norm is: 166.3285383865618
total time for 1 cost function evaluation is 0.17364096641540527

Initializing K-matrix took: 0.016102313995361328
exponentiating matrix took: 0.003545045852661133
Matrix multiplication took: 2.2649765014648438e-05
extracting integrals took: 0.13318395614624023
calculating norm of qubit hamiltonian took 0.017444372177124023
1-norm is: 166.13540740433234
total time for 1 cost function evaluation is 0.17155814170837402

Initializing K-matrix took: 0.017379283905029297
exponentiating matrix took: 0.00330686

extracting integrals took: 0.11763572692871094
calculating norm of qubit hamiltonian took 0.019231081008911133
1-norm is: 165.1205648503092
total time for 1 cost function evaluation is 0.1570265293121338

Initializing K-matrix took: 0.015506744384765625
exponentiating matrix took: 0.0033507347106933594
Matrix multiplication took: 2.3126602172851562e-05
extracting integrals took: 0.11635088920593262
calculating norm of qubit hamiltonian took 0.017513036727905273
1-norm is: 177.3268311355078
total time for 1 cost function evaluation is 0.15401339530944824

Initializing K-matrix took: 0.015599489212036133
exponentiating matrix took: 0.003293752670288086
Matrix multiplication took: 2.4557113647460938e-05
extracting integrals took: 0.11928510665893555
calculating norm of qubit hamiltonian took 0.018414974212646484
1-norm is: 169.25777150759862
total time for 1 cost function evaluation is 0.15790009498596191

Initializing K-matrix took: 0.01736140251159668
exponentiating matrix took: 0.00334

extracting integrals took: 0.11756587028503418
calculating norm of qubit hamiltonian took 0.017750024795532227
1-norm is: 166.379277079606
total time for 1 cost function evaluation is 0.15551042556762695

Initializing K-matrix took: 0.015545845031738281
exponentiating matrix took: 0.003313302993774414
Matrix multiplication took: 2.3365020751953125e-05
extracting integrals took: 0.1162421703338623
calculating norm of qubit hamiltonian took 0.018182992935180664
1-norm is: 163.52806674180835
total time for 1 cost function evaluation is 0.15455412864685059

Initializing K-matrix took: 0.015741348266601562
exponentiating matrix took: 0.003340005874633789
Matrix multiplication took: 2.3365020751953125e-05
extracting integrals took: 0.11918878555297852
calculating norm of qubit hamiltonian took 0.018395423889160156
1-norm is: 165.43303085173636
total time for 1 cost function evaluation is 0.1579296588897705

Initializing K-matrix took: 0.015776395797729492
exponentiating matrix took: 0.003388

...

extracting integrals took: 0.12092399597167969
calculating norm of qubit hamiltonian took 0.01778435707092285
1-norm is: 140.6404299413285
total time for 1 cost function evaluation is 0.15887045860290527

Initializing K-matrix took: 0.01553964614868164
exponentiating matrix took: 0.003301858901977539
Matrix multiplication took: 2.3365020751953125e-05
extracting integrals took: 0.11863398551940918
calculating norm of qubit hamiltonian took 0.017508983612060547
1-norm is: 140.640433359059
total time for 1 cost function evaluation is 0.15623879432678223

Initializing K-matrix took: 0.015650033950805664
exponentiating matrix took: 0.0033469200134277344
Matrix multiplication took: 2.288818359375e-05
extracting integrals took: 0.11568832397460938
calculating norm of qubit hamiltonian took 0.01804518699645996
1-norm is: 140.64040240827313
total time for 1 cost function evaluation is 0.1539936065673828

Initializing K-matrix took: 0.015482902526855469
exponentiating matrix took: 0.003279685974

extracting integrals took: 0.1156160831451416
calculating norm of qubit hamiltonian took 0.017558574676513672
1-norm is: 140.64038986816482
total time for 1 cost function evaluation is 0.15327239036560059

Initializing K-matrix took: 0.015465974807739258
exponentiating matrix took: 0.0032889842987060547
Matrix multiplication took: 2.288818359375e-05
extracting integrals took: 0.1150519847869873
calculating norm of qubit hamiltonian took 0.017910242080688477
1-norm is: 140.6403382178849
total time for 1 cost function evaluation is 0.1529829502105713

Initializing K-matrix took: 0.015605449676513672
exponentiating matrix took: 0.00333404541015625
Matrix multiplication took: 2.1696090698242188e-05
extracting integrals took: 0.11588191986083984
calculating norm of qubit hamiltonian took 0.01785588264465332
1-norm is: 140.64043274602443
total time for 1 cost function evaluation is 0.15393972396850586

Initializing K-matrix took: 0.015613079071044922
exponentiating matrix took: 0.00327706336

extracting integrals took: 0.11562585830688477
calculating norm of qubit hamiltonian took 0.017974853515625
1-norm is: 140.6403661048444
total time for 1 cost function evaluation is 0.15370726585388184

Initializing K-matrix took: 0.015619516372680664
exponentiating matrix took: 0.0033011436462402344
Matrix multiplication took: 2.3365020751953125e-05
extracting integrals took: 0.1152646541595459
calculating norm of qubit hamiltonian took 0.017897367477416992
1-norm is: 140.64032194968715
total time for 1 cost function evaluation is 0.15334081649780273

Initializing K-matrix took: 0.015506505966186523
exponentiating matrix took: 0.00328826904296875
Matrix multiplication took: 2.288818359375e-05
extracting integrals took: 0.11563301086425781
calculating norm of qubit hamiltonian took 0.018021583557128906
1-norm is: 140.64033538105414
total time for 1 cost function evaluation is 0.15371155738830566

Initializing K-matrix took: 0.015590190887451172
exponentiating matrix took: 0.00329327583

extracting integrals took: 0.11894059181213379
calculating norm of qubit hamiltonian took 0.01760721206665039
1-norm is: 140.6403594933611
total time for 1 cost function evaluation is 0.15679335594177246

Initializing K-matrix took: 0.0157320499420166
exponentiating matrix took: 0.0034291744232177734
Matrix multiplication took: 2.3603439331054688e-05
extracting integrals took: 0.11753225326538086
calculating norm of qubit hamiltonian took 0.01784372329711914
1-norm is: 140.64028158520514
total time for 1 cost function evaluation is 0.1558094024658203

Initializing K-matrix took: 0.01570725440979004
exponentiating matrix took: 0.0033152103424072266
Matrix multiplication took: 2.384185791015625e-05
extracting integrals took: 0.12046504020690918
calculating norm of qubit hamiltonian took 0.017439842224121094
1-norm is: 140.6403238690504
total time for 1 cost function evaluation is 0.1581897735595703

Initializing K-matrix took: 0.015636920928955078
exponentiating matrix took: 0.0033226013

extracting integrals took: 0.11680030822753906
calculating norm of qubit hamiltonian took 0.018000125885009766
1-norm is: 140.64028455593706
total time for 1 cost function evaluation is 0.1550748348236084
total time for minimization with 50000 max iterations was: 458.55862832069397


In [4]:
print("message:",f_min_OO.message,"number of function evaluations:",f_min_OO.nfev)

message: Optimization terminated successfully. number of function evaluations: 2887


In [12]:
K = K_matr(f_min_OO.x, active_indices)
U_OO   = scipy.linalg.expm( - K )
if localize:
    C_OO = C_locPM @ U_OO
else:
    C_OO = C_nonloc @ U_OO

one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_OO, threshold)
if consider_cas:
        CASconstant, one_body_integrals, two_body_integrals =\
        md.get_active_space_integrals(one_body_integrals,
                                      two_body_integrals,
                                      occupied_indices,
                                      active_indices)
else:
        CASconstant = 0
OneNormorbOO = JW1norm_spat(np.copy(constant+CASconstant),
                            np.copy(one_body_integrals),
                            np.copy(two_body_integrals))

one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_nonloc, threshold)
if consider_cas:
        CASconstant, one_body_integrals, two_body_integrals =\
        md.get_active_space_integrals(one_body_integrals,
                                      two_body_integrals,
                                      occupied_indices,
                                      active_indices)
else:
        CASconstant = 0
OneNormorbnonloc = JW1norm_spat(np.copy(constant+CASconstant),
                                np.copy(one_body_integrals),
                                np.copy(two_body_integrals))

one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_locPM, threshold)
if consider_cas:
        CASconstant, one_body_integrals, two_body_integrals =\
        md.get_active_space_integrals(one_body_integrals,
                                      two_body_integrals,
                                      occupied_indices,
                                      active_indices)
else:
        CASconstant = 0
OneNormorblocPM = JW1norm_spat(np.copy(constant+CASconstant),
                       np.copy(one_body_integrals),
                       np.copy(two_body_integrals))
if randomize:
    print("Starting from random orbital rotation...")
if consider_cas:
    print("Considering CAS(" + str(n_electrons) + "," + str(n_orbitals) + "), \n1norm of CMO orb is",
         OneNormorbnonloc, "\n1norm of PM orb is", OneNormorblocPM, "\n1norm of optimizer is", OneNormorbOO)
else:
    print("Considering full space, \n1norm of CMO orb is",
         OneNormorbnonloc, "\n1norm of PM orb is", OneNormorblocPM, "\n1norm of optimizer is", OneNormorbOO)

Starting from random orbital rotation...
Considering CAS(12,12), 
1norm of CMO orb is 163.41136361700745 
1norm of PM orb is 144.0570143662687 
1norm of optimizer is 140.64028455593706


## Visualize
If you want to save the orbitals to gaussian CUBE files

In [14]:
Cost_function_OO_OneNorm(f_min_OO.x)
K = K_matr(f_min_OO.x, active_indices)
U_OO   = scipy.linalg.expm( - K )
if localize:
    C_OO = C_locPM @ U_OO
else:
    C_OO = C_nonloc @ U_OO
t13 = time.time()
for i in range(ncore,ntot):    
    tools.cubegen.orbital(mol, os.getcwd() + '/CUBE_FILES/pyscfcube'\
                          + description + 'onenorm_orb' + localizemethod + str(localize)\
                          + str(randomize) + str(consider_cas) + str(i) , C_OO[:,i])
print('Cube files of molecule', description,'created in', os.getcwd() + '/CUBE_FILES/')
print('extracting cube files took', time.time()-t13)


Initializing K-matrix took: 0.023717880249023438
exponentiating matrix took: 0.0028357505798339844
Matrix multiplication took: 0.00014448165893554688
extracting integrals took: 0.09226322174072266
calculating norm of qubit hamiltonian took 0.018866300582885742
1-norm is: 140.64028455593706
total time for 1 cost function evaluation is 0.13923192024230957
Cube files of molecule hnch2_s0min_dzpcc-pvdzne12no12 created in /home/emielkoridon/1-Norm_calculations/CUBE_FILES/
extracting cube files took 4.6578898429870605
